In [14]:
import json
import numpy as np
from bias_explorer.operations import report
from bias_explorer.utils import system, dataloader
import glob
import pandas as pd

In [2]:
predictions_path = "./data/fairface/predictions/openCLIP/ViT-L-14/commonpool_xl_s13b_b90k/age_race_gender/"

In [3]:
def get_topk_preds(preds_path):
    preds_list = glob.glob(predictions_path + 'top*.csv')
    k_preds = {}
    for pred in preds_list:
        fname = pred.split("/")[-1]
        name_parts = fname.split("_")
        if len(name_parts) > 2:
            k = name_parts[1]
        else:
            k = 1
        k_preds[k] = pred
    return k_preds

In [4]:
avg_path = predictions_path + "sum_synms.csv"
mode_name = "Avg Sum"
metric_name = "accuracy"
sum_df = dataloader.load_df(avg_path)

In [5]:
def get_col_list(df, metric_name):
    if metric_name == "balanced_accuracy":
        col_list = list(df.drop(
            columns=['file', 'gender_preds', 'gender']).keys())
    else:
        col_list = list(df.drop(columns=['file', 'gender_preds']).keys())
        col_list = system.list_item_swap(col_list, 'age', 'gender')
    col_list = system.list_item_swap(col_list, 'age', 'race')
    return col_list

In [6]:
def get_uniques(df, col_list):
    uniques = []
    for col in col_list:
        col_items = list(df[col].unique())
        if col == "age":
            col_items = system.fix_age_order(col_items)
        for unique in col_items:
            if col == "age" and unique == "00-02":
                unique = "0-2"
            elif col == "age" and unique == "03-09":
                unique = "3-9"
            uniques.append(unique)
    return uniques

In [7]:
def get_empty_report_dict(df, metric_name):
    report_dict = {
        'Mode': [],
        metric_name: [],
    }

    col_list = get_col_list(df, metric_name)
    uniques = get_uniques(df, col_list)
    for unique in uniques:
        report_dict[unique] = []
    return report_dict

In [8]:
# gen_dict_report
def gen_dict_report(df, mode_name, metric_name, rep_dict):
    metric_func = report.metric_loader(metric_name)
    rep_dict['Mode'].append(mode_name)
    rep_dict[metric_name].append(report.gender_eval(df, metric_func))
    col_list = get_col_list(df, metric_name)
    for col in col_list:
        uniques = get_uniques(df, [col])
        for unique in uniques:
            col_df = report.filter_df(df, col, unique)
            col_acc = report.gender_eval(col_df, metric_func)
            rep_dict[unique].append(col_acc)
    return rep_dict

In [9]:
rep_dict = get_empty_report_dict(sum_df, metric_name)
rep_dict = gen_dict_report(sum_df, 'Avg Sum', 'accuracy', rep_dict)
rep_dict

{'Mode': ['Avg Sum'],
 'accuracy': [0.9582],
 'Male': [0.9356],
 'Female': [0.9835],
 'East Asian': [0.949],
 'White': [0.965],
 'Latino_Hispanic': [0.9698],
 'Southeast Asian': [0.9428],
 'Black': [0.9319],
 'Indian': [0.969],
 'Middle Eastern': [0.981],
 '0-2': [0.7085],
 '3-9': [0.8975],
 '10-19': [0.9441],
 '20-29': [0.9688],
 '30-39': [0.9845],
 '40-49': [0.9815],
 '50-59': [0.9724],
 '60-69': [0.9907],
 'more than 70': [0.9492]}

In [10]:
k_preds = get_topk_preds(predictions_path)

In [13]:
print(k_preds)

{'2': './data/fairface/predictions/openCLIP/ViT-L-14/commonpool_xl_s13b_b90k/age_race_gender/top_2_synms.csv', '1': './data/fairface/predictions/openCLIP/ViT-L-14/commonpool_xl_s13b_b90k/age_race_gender/top_1_synms.csv'}


In [11]:
for k, path in k_preds.items():
    mode_name = f"Top {k}"
    k_df = dataloader.load_df(path)
    rep_dict = gen_dict_report(k_df, mode_name, metric_name, rep_dict)

In [15]:
rep_df = pd.DataFrame(rep_dict)
rep_df.head()

,Mode,accuracy,Male,Female,East Asian,White,Latino_Hispanic,Southeast Asian,Black,Indian,Middle Eastern,0-2,3-9,10-19,20-29,30-39,40-49,50-59,60-69,more than 70
0,Avg Sum,0.9582,0.9356,0.9835,0.9490,0.9650,0.9698,0.9428,0.9319,0.9690,0.981,0.7085,0.8975,0.9441,0.9688,0.9845,0.9815,0.9724,0.9907,0.9492
1,Top 2,0.9609,0.9498,0.9735,0.9561,0.9583,0.9692,0.9512,0.9415,0.9736,0.981,0.7739,0.9211,0.9483,0.9658,0.9815,0.9808,0.9711,0.9875,0.9492
2,Top 1,0.9603,0.9501,0.9717,0.9535,0.9583,0.9686,0.9534,0.9396,0.9723,0.981,0.7538,0.9181,0.9483,0.9645,0.9820,0.9830,0.9724,0.9875,0.9492


In [3]:
from bias_explorer.operations import report
from bias_explorer.utils import dataloader, system

In [6]:
conf = dataloader.load_config()
conf['DataSource'] = 'openai'
metric_name = conf['Metric']
preds = system.concat_out_path(conf, 'Predictions')
print(preds)

./data/fairface/predictions/openCLIP/ViT-L-14/openai/age_race_gender
